In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import logging
import numpy as np
import os

logger = logging.getLogger(__name__)

In [ ]:
def extract_metric_across_steps(df, metric_base_name, steps):
    all_metric_values = []
    num_iterations = len(df)
    for i in range(num_iterations):
        iteration_row = df.iloc[i]
        for step_name in steps:
            col_name = f"{step_name}_{metric_base_name}"
            if col_name in df.columns:
                value = iteration_row[col_name]
                if pd.isna(value):
                    all_metric_values.append(np.nan)
                else:
                    all_metric_values.append(value)
            else:
                logger.warning(f"Column '{col_name}' not found for iteration {i}.")
                all_metric_values.append(np.nan)
    if not all_metric_values:
        logger.error(f"Could not extract any data for metric '{metric_base_name}'.")
        return None, None
    metric_array = np.array(all_metric_values, dtype=float)
    x_axis = np.arange(len(metric_array))
    return metric_array, x_axis

In [ ]:
def plot_interleaved_metrics(df, steps=None, metrics=None):
    if steps is None:
        steps = ["washtrade", "claim_fees", "remove_liquidity"]
    if metrics is None:
        metrics = [
            ("pool_virtual_price", "tab:blue", "-", "Virtual Price"),
            ("pool_xcp_profit", "tab:orange", "--", "XCP Profit"),
            ("pool_xcp_profit_a", "tab:red", "--", "XCP Profit"),
        ]
    fig, ax = plt.subplots(figsize=(14, 8))
    max_iter = len(df)
    x_points = []
    x_labels = []
    all_values = {metric_name: [] for metric_name, _, _, _ in metrics}
    for iter_idx in range(max_iter):
        for step in steps:
            x_idx = len(x_points)
            x_points.append(x_idx)
            short_step = step[:4] if step != "claim_fees" else "clm"
            x_labels.append(f"{short_step}({iter_idx})")
            for metric_name, _, _, _ in metrics:
                col_name = f"{step}_{metric_name}"
                if col_name in df.columns:
                    value = df.iloc[iter_idx][col_name]
                    all_values[metric_name].append(value)
                else:
                    all_values[metric_name].append(np.nan)
    for metric_name, color, linestyle, label in metrics:
        ax.plot(
            x_points,
            all_values[metric_name],
            label=label,
            color=color,
            linestyle=linestyle,
            alpha=0.8,
        )
    ax.set_ylabel("Metric Value")
    ax.set_title("Pool Metrics Across Sequential Steps")
    ax.grid(True, linestyle="-", alpha=0.7)
    ax.legend(loc="best")
    fig.tight_layout()
    plt.show()

In [ ]:
def plot_remove_liquidity_total_values(df):
    fig, ax = plt.subplots(figsize=(12, 6))
    step = "remove_liquidity"
    actors = [
        ("lp_user", "tab:blue", "LP"),
        ("trader", "tab:orange", "Trader"),
        ("admin", "tab:green", "Admin"),
    ]
    total = np.zeros(len(df))
    for actor, color, label in actors:
        col = f"{step}_{actor}_total_value"
        ax.plot(df["iteration"], df[col], label=label, color=color, alpha=0.8)
        total += df[col]
    ax.plot(df["iteration"], total, label="Total", color="tab:red", alpha=0.8)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Total Value (in coin0 terms)")
    ax.set_title("Total Value at remove_liquidity Step")
    ax.grid(True, linestyle="--", alpha=0.7)
    ax.legend(loc="best")
    fig.tight_layout()
    plt.show()

In [ ]:
# Example usage (run in a separate cell after loading your DataFrame):
cur_dir = os.path.dirname(os.path.abspath(".")) + "/simulator/"
csv_filename = os.path.join(cur_dir, "pool_simulation_results1.csv")
df = pd.read_csv(csv_filename)
steps = ["washtrade", "claim_fees", "remove_liquidity"]
plot_interleaved_metrics(df, steps)
plot_remove_liquidity_total_values(df)